<a href="https://colab.research.google.com/github/Eliascc5/English_proficiency_prediction_NLP/blob/main/projet_IAS_partie2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#As in the first stage -> we load our dataset from Drive 
#Dataset: NICT_JLE_4.1
#Reference: https://alaginrc.nict.go.jp/nict_jle/index_E.html

from google.colab import drive
drive.mount("/content/gdrive")

#----------------------------------


Mounted at /content/gdrive


In [7]:
import os

path_test= r'/content/gdrive/MyDrive/NICT_JLE_4.1/Output/'
os.chdir(path_test)

vocabulary=[]
y_output=[]

for file in os.listdir():
  if file.endswith(".txt"):

    file_path = f"{path_test}/{file}"

    with open(file_path, mode='r',encoding="utf8",errors='ignore') as f:

      score = f.readline()
      content = f.read()

      vocabulary.append(content)

      y_output.append(int(score))

      f.close

######################################

**NOTE:** We create a general vocabulary from the transcription of all the candidates so that it is as general as possible. 

Basically the "vocabulary" list contains in each element the transcription of each candidate.  In other words -> len (vocabulary) = 1281 

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer() 
# tokenize and build vocab
vectorizer.fit(vocabulary)  #(Learn a vocabulary dictionary of all tokens in the raw documents.)
# summarize
#print(vectorizer.vocabulary_)

print(vectorizer.get_feature_names())

# encode document
vector = vectorizer.transform(vocabulary)
# summarize encoded vector

#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

['aa', 'aaa', 'aaah', 'aaanto', 'aah', 'aahh', 'aahn', 'aan', 'aanto', 'aato', 'aatone', 'aatto', 'ab', 'aba', 'abandoned', 'abba', 'abe', 'abilities', 'ability', 'able', 'abnormal', 'abo', 'aboard', 'abou', 'abound', 'about', 'above', 'abraod', 'abro', 'abroad', 'absence', 'absent', 'absolutely', 'absorb', 'absorbed', 'abstract', 'abuse', 'abused', 'abut', 'ac', 'aca', 'academic', 'academical', 'academy', 'acce', 'accelerate', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'accepting', 'accepts', 'access', 'accessibility', 'accessible', 'accessing', 'accessories', 'accessory', 'acci', 'accide', 'acciden', 'accident', 'accidental', 'accidentally', 'accidents', 'acco', 'accommodate', 'accommodation', 'accommodations', 'accompanied', 'accompaniment', 'accompany', 'accomplished', 'according', 'accordingly', 'accordion', 'accoun', 'account', 'accountant', 'accountantment', 'accounting', 'accountings', 'accountist', 'accountment', 'accounts', 'accuding', 'accurate', 'accurately', 

**Here we start with the Neural Network**


In [ ]:
from keras import models
from keras import layers
from keras.utils import np_utils
import tensorflow as tf
import numpy as np

#######################################
n = len(vocabulary)   #Number of candidates 1281
m = 15157  #Number of words without repetition of our vocabulary 

array=vector.toarray()                      #INPUT

array_output = np.array(y_output)           #OUTPUT

array_output = np_utils.to_categorical(array_output) 

array_output = array_output[:,1:10]


########################################

model = models.Sequential()
model.add(layers.Dense(200, activation='sigmoid', input_shape=(m,)))
model.add(layers.Dense(400, activation='sigmoid'))
model.add(layers.Dense(200, activation='sigmoid'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



######################################################################

#Repartition des donnees
randomGenerator = np.random.RandomState(0) #Generateur de nombres aleatoires
nbIndividus = len(vocabulary) #Nombre d'individus presents dans la base de donnees
randomIndexes = np.arange(nbIndividus)#Creation d'une liste d'index
randomGenerator.shuffle(randomIndexes)#Melange de cette liste
#Utilisation de cette liste d'index pour repartir les donnees en differents jeux
#Apprentissage : 60%
x_train_array = array[randomIndexes[0:int(len(vocabulary)*0.6)]]
y_train_array = array_output[randomIndexes[0:int(len(vocabulary)*0.6)]]
#Validation : 20 %
x_val_array = array[randomIndexes[int(len(vocabulary)*0.6):int(len(vocabulary)*0.8)]]
y_val = array_output[randomIndexes[int(len(vocabulary)*0.6):int(len(vocabulary)*0.8)]]
#Test : 20%
x_test_array = array[randomIndexes[int(len(vocabulary)*0.8):]]
y_test = array_output[randomIndexes[int(len(vocabulary)*0.8):]]


history = model.fit(x_train_array,
                    y_train_array,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val_array, y_val))

#plot the results :
history_dict = history.history

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


**TODO:** add prediction stage; Network performance tests   **IN PROCESS**

In [37]:
##IN PROCESS....
predictions= model.predict(x_test_array) 

for i in range(10):
  print("Prediction: ", np.argmax(predictions[i])+1)  #add +1 to start in 1
  print("Label of x_test", y_test[i])
  print("------------------")

Prediction:  4
Label of x_test [0. 0. 0. 1. 0. 0. 0. 0. 0.]
------------------
Prediction:  5
Label of x_test [0. 0. 0. 0. 0. 1. 0. 0. 0.]
------------------
Prediction:  4
Label of x_test [0. 0. 0. 1. 0. 0. 0. 0. 0.]
------------------
Prediction:  4
Label of x_test [0. 0. 0. 1. 0. 0. 0. 0. 0.]
------------------
Prediction:  5
Label of x_test [0. 0. 0. 0. 1. 0. 0. 0. 0.]
------------------
Prediction:  3
Label of x_test [0. 0. 1. 0. 0. 0. 0. 0. 0.]
------------------
Prediction:  4
Label of x_test [0. 0. 0. 0. 1. 0. 0. 0. 0.]
------------------
Prediction:  3
Label of x_test [0. 0. 0. 1. 0. 0. 0. 0. 0.]
------------------
Prediction:  5
Label of x_test [0. 0. 0. 0. 0. 0. 1. 0. 0.]
------------------
Prediction:  7
Label of x_test [0. 0. 0. 0. 0. 0. 0. 1. 0.]
------------------
